In [1]:
%load_ext pyflame

In [2]:
# setup
from __future__ import annotations
from time import time
import random
from collections import defaultdict
# get the words
from nltk.corpus import words
from english_words import english_words_lower_set 
import numpy as np
import pandas as pd
import string

# fivers = [word for word in words.words() if len(word)==5]
fivers = [word for word in english_words_lower_set if len(word)==5 and set(word) < set(string.ascii_lowercase)]
print(len(fivers))

3194


In [70]:
def get_letter_freqs(words):
    freqs = defaultdict(int)
#     letterstring = "".join(words)
#     for letter in string.ascii_lowercase:
#         freqs[letter] += letterstring.count(letter)
        
    for word in words:
        for letter in set(word):
            freqs[letter]+=1
    return freqs

def letter_freq_sorted(
    possibe_words,
    letter_freqs):

    # print([(k,v) for k,v in sorted(letter_freqs.items(), key = lambda pair: pair[1])])
    return sorted(possibe_words, key = lambda w: sum([letter_freqs[l] for l in set(w)]), reverse=True)
    
freq_sorted_fivers = letter_freq_sorted(fivers, get_letter_freqs(fivers))
print(freq_sorted_fivers[:10])

['arose', 'orate', 'erato', 'arise', 'raise', 'aries', 'aires', 'lares', 'irate', 'artie']


In [76]:
def get_word() -> str:
    return random.choice(fivers)

# returns a list evaluating guess by index. 
# 0 = not in word
# 1 = in word, wrong loc
# 2 = in word, right loc
def check_word(guess: str, word: str) -> list[int]:
    assert len(guess) == len(word) == 5, f"(guess, word) = ({guess}, {word})"
    check = [0]*5
    for i in range(len(guess)):
        if guess[i] not in word:
            continue
        check[i]+=1
        if guess[i] == word[i]:
            check[i]+=1
            
    return check

def get_prev_guess_info(
    past_guesses: list[str], 
    past_checks: list[list[int]]) -> tuple[list, list, set]:

    past_yellow = [] #list{idx:char}, equiv to yellows
    yellow = [] #list{idx:char}, equiv to yellows which have gone green
    green = [] #list{idx:char}, equiv to greens
    contained = set() #yellow + green letters
    not_contained = set() # bad guesses
    
    for g in range(len(past_guesses)):
        for idx in range(len(past_guesses[g])):
            letter = past_guesses[g][idx]
            check = past_checks[g][idx]
            if check == 0:
                not_contained.add(letter)
            else:
                contained.add(letter)
            if check >= 1:
                past_yellow.append((idx, letter))
            if check == 2:
                green.append((idx, letter))
    green_letters = set(l for i,l in green)
    for idx, l in past_yellow:
        if l not in green_letters:
            yellow.append((idx, l))
    # print(green_letters)
    # print(yellow)
    return green, yellow, contained, not_contained

def generate_guess(
    past_guesses: list[str], 
    past_checks: list[list[int]],
    word_set: list[str],
    possible_words: list[str],
    novel_words: list[str]) -> str:
    
    if len(past_guesses) == 0:
        return freq_sorted_fivers[0]
    
    green, yellow, contained, not_contained = get_prev_guess_info(past_guesses, past_checks)
    
    guessed_letters = contained.union(not_contained)

    def no_invalid_letters_filt(word):
        wset = set(word)
        if not wset.isdisjoint(not_contained): # don't reuse known bad letters
            return False
    
    def novelty_filt(word):
        wset = set(word)
        return wset.isdisjoint(guessed_letters)
    
    def filt(word):
        # fail words without placed greens
        for idx, l in green:
            if word[idx] != l:
                return False
        # fail words which don't contain the known letters in the word
        wset = set(word)
        if not wset.isdisjoint(not_contained): # don't reuse known bad letters
            return False
        if not wset >= contained:
            return False
        # fail words which contain yellows in the same position
        for idx, l in yellow:
            if word[idx] == l:
                return False
        
        if word in (past_guesses):
            return False
        
        return True
    
    # filter possible_words in place to reduce list size each round
    for idx in range(len(possible_words))[::-1]:
        if not filt(possible_words[idx]):
            possible_words.pop(idx)
    possible_words_freqs = get_letter_freqs(possible_words)
          
    if (len(possible_words) > 1
        and len(contained)<5 # in this case, we've found all the letters
        ): 
        for idx in range(len(novel_words))[::-1]:
            if not novelty_filt(novel_words[idx]):
                novel_words.pop(idx)
        
        freq_sorted_common_novel = letter_freq_sorted(novel_words, get_letter_freqs(possible_words))
        if len(freq_sorted_common_novel) > 0:
            for novel_word in freq_sorted_common_novel:
                if len(set(novel_word))==5:
                    return novel_word
        else:
            unknown_letter_freqs = {}
            for l in string.ascii_lowercase:
                if l in guessed_letters:
                    unknown_letter_freqs[l] = 0
                else:
                    unknown_letter_freqs[l] = possible_words_freqs[l]
            freq_sorted_non_novel = letter_freq_sorted(filter(no_invalid_letters_filt, fivers), unknown_letter_freqs)
            for non_novel_word in freq_sorted_non_novel:
                if non_novel_word not in past_guesses:
                    return non_novel_word
    if len(possible_words) >= 3 and len(green) == 4:
        possible_unguessed_freqs = {}
        for l in set(string.ascii_lowercase):
            if l in guessed_letters:
                possible_unguessed_freqs[l]=0
            else:
                possible_unguessed_freqs[l]=possible_words_freqs[l]
        if sum(possible_unguessed_freqs.values()) > 0:
            freq_sorted_unguessed = letter_freq_sorted(fivers, possible_unguessed_freqs)
            for unguessed_letter_word in freq_sorted_unguessed:
                return unguessed_letter_word
                    
    freq_sorted_possible = letter_freq_sorted(possible_words, possible_words_freqs)

    # return word with most unique letters
    return sorted(freq_sorted_possible, key = lambda w: len(set(w)), reverse = True)[0]

# recursion, whee!
def guess_until_right(
    hidden_word: str,
    past_guesses: list[str], 
    past_checks: list[list[int]],
    word_set: list[str], # still needed?
    possible_words: list[str],
    novel_words: list[str]) -> tuple[str,list,list]:

    current_guess = generate_guess(past_guesses, past_checks, word_set, possible_words, novel_words)
    current_check = check_word(current_guess, hidden_word)
    past_guesses.append(current_guess)
    past_checks.append(current_check)
    
    if sum(current_check) == 10:
        return hidden_word, past_guesses, past_checks
    else:
        return guess_until_right(hidden_word, past_guesses, past_checks, word_set, possible_words, novel_words)
    
def play_game(hidden_word: str) -> tuple[str,list,list]:
    possible_words = [word for word in freq_sorted_fivers]
    novel_words = [word for word in freq_sorted_fivers]
    return guess_until_right(hidden_word, [], [], fivers, possible_words, novel_words)
    

In [77]:
%%pyflame
start = time()
rounds = 200
tries = 0
for _ in range(rounds):
    word, guesses, checks = play_game(get_word())
    tries += len(guesses)
print(f"finished {rounds} rounds in {time()-start}s, avergaging {tries/rounds} guesses/round")

finished 200 rounds in 0.7169415950775146s, avergaging 3.94 guesses/round


In [73]:
%%pyflame

rounds = 2500
tries_arr = np.ndarray((rounds, 1))

start = time()
tries = 0
for rnd in range(rounds):
    word, guesses, checks = play_game(get_word())
    tries += len(guesses)
    tries_arr[rnd, 0] = len(guesses)
    if len(guesses)>7:
        print(guesses)
    # print(f"guessed {w} in {len(gs)} tries")
print(f"finished {rounds} rounds in {time()-start}s, avergaging {tries/rounds} guesses/round")

['arose', 'blunt', 'hippy', 'cater', 'drawn', 'month', 'eater', 'rater', 'tater']
finished 2500 rounds in 9.484548807144165s, avergaging 3.9612 guesses/round


In [81]:
%%pyflame

tries_list=[]

start = time()
tries = 0
for hidden_word in fivers:
    word, guesses, checks = play_game(hidden_word)
    tries += len(guesses)
    tries_list.append(len(guesses))
    if len(guesses)>6:
        print(guesses)
print(f"finished all words in {time()-start}s, avergaging {tries/len(fivers)} guesses/round")

['arose', 'until', 'colby', 'moldy', 'holly', 'golly', 'folly', 'jolly']
['arose', 'tulip', 'stake', 'caste', 'haste', 'baste', 'waste']
['arose', 'blunt', 'picky', 'earth', 'dater', 'mater', 'water', 'rater']
['arose', 'until', 'colby', 'moldy', 'holly', 'golly', 'folly', 'jolly', 'lolly']
['arose', 'unity', 'decry', 'plumb', 'flung', 'kajar', 'veery']
['arose', 'until', 'colby', 'moldy', 'holly', 'golly', 'folly']
['arose', 'unity', 'bumpy', 'lucky', 'dully', 'gully', 'fully']
['arose', 'unity', 'ditch', 'light', 'might', 'fight', 'tight']
['arose', 'blunt', 'picky', 'earth', 'dater', 'mater', 'water', 'rater', 'tater']
['arose', 'unity', 'decry', 'plumb', 'flung', 'kajar', 'kerry']
['arose', 'tunic', 'lymph', 'drill', 'grill', 'krill', 'frill']
['arose', 'unity', 'lusty', 'dusty', 'musty', 'gusty', 'fusty']
['arose', 'unity', 'decry', 'plumb', 'flung', 'kajar', 'jerry']
['arose', 'blunt', 'picky', 'earth', 'dater', 'mater', 'water']
finished all words in 11.164659023284912s, avergag

In [82]:
pd.DataFrame(tries_list).describe()

,0
count,3194.000000
mean,3.924546
std,0.806879
min,1.000000
25%,3.000000
50%,4.000000
75%,4.000000
max,9.000000
